In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

## Load Dataset

In [ ]:

stock = yf.Ticker('TATAMOTORS.NS')
data = stock.history(period = 'max')
data

In [ ]:
close = data['Close'].values
close=close.reshape(-1,1)
close.shape

## Prepare Dataset

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))
scaled_data.shape

In [ ]:
scaled_data

In [ ]:
train_data = scaled_data[:].reshape(-1,1)
train_data.shape

In [ ]:
prediction_days = 100 # look back for next day 120,123,124,125,126,127 =====> [120,123,124] -> [125] , [123,124,125] -> [126]

In [ ]:
x_train = []
y_train = []

In [ ]:
for x in range(prediction_days, len(train_data)):
  x_train.append(train_data[x-prediction_days:x, 0])
  y_train.append(train_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
y_train.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

## Build Model

In [ ]:
model = Sequential()
model.add(LSTM(units = 100, return_sequences=True, input_shape = (x_train.shape[1],1)))

model.add(LSTM(units=100, return_sequences=True))

model.add(LSTM(units=100))

model.add(Dense(units = 1, activation = 'relu'))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs = 100, batch_size = 64)

In [ ]:
pred = model.predict(x_train)

In [ ]:
pred.shape

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_train, label = "Training Data")
plt.plot(pred, label = 'Model 1 Prediction')
plt.legend()
plt.show()

## Test the Accuracy

In [ ]:
#Load Test Data

In [ ]:
stock = yf.Ticker('SBIN.NS')
data = stock.history(period = 'max')
data

In [ ]:
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))
test_data = scaled_data[:].reshape(-1,1)
test_data.shape

In [ ]:
x_test = []
y_test = []
prediction_days = 100

In [ ]:
for x in range(prediction_days, len(test_data)):
  x_test.append(test_data[x-prediction_days:x, 0])
  y_test.append(test_data[x, 0])

x_test, y_test = np.array(x_test), np.array(y_test)

In [ ]:
x_test.shape, y_test.shape

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

## Make Predictions

In [ ]:
model.summary()

In [ ]:
prediction_price = model.predict(x_test)

In [ ]:
y_test = scaler.inverse_transform(y_test)
prediction_price = scaler.inverse_transform(prediction_price)

In [ ]:
prediction_price.shape, y_test.shape

## Plot the predictions

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_test, color = 'red', label = 'Actual')
plt.plot(prediction_price, color = 'blue', label = 'LSTM Prediction')
plt.ylabel(f'{code.split('.')[0]} Price in Rs')
plt.xlabel('Time in Integer')
plt.title(f'{code.split('.')[0]} Share Prices')
plt.legend()
plt.show()

## Future price

In [ ]:
real_data = [model_inputs[len(model_inputs)-prediction_days:len(model_inputs), 0]]
real_data = np.array(real_data)

In [ ]:
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))
real_data.shape

(1, 100, 1)

In [ ]:
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)

In [ ]:
print(f'Next day price of {code} Stock is : {prediction[0][0]}')

Next day price of HINDALCO.NS Stock is : 313.9100646972656


## Errors

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
actual_price = actual_price.reshape(-1,1)
actual_price.shape, prediction_price.shape

In [ ]:
y_train = y_train.reshape(-1,1)
y_train.shape, pred.shape

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train, pred))
rmse_test = np.sqrt(mean_squared_error(actual_price, prediction_price))

In [ ]:
rmse_train , rmse_test

In [ ]:
mae_train = mean_absolute_error(y_train, pred)
mae_train

In [ ]:
mae_test = mean_absolute_error(actual_price, prediction_price)
mae_test

In [ ]:
r2_train = r2_score(y_train, pred)
r2_train

In [ ]:
r2_test = r2_score(actual_price, prediction_price)
r2_test